## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2024xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from tab_dataset.cdataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib

def analyse_integrite(data, schema, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le schéma 'schema'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(schema)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    #'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
    # 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
    # log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    #'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  98677


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         #{"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" }
    ]
}
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 4 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  1939 soit :  0.01964996909107492
nombre de pdc en itinerance   :  96738


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(itinerance_init.npd.analysis().tree())

-1: root-derived (96738)
   0 : index (0 - 96738)
   1 : contact_operateur (96571 - 167)
   2 : nom_enseigne (93587 - 3151)
   3 : coordonneesXY (70480 - 26258)
   4 : adresse_station (70248 - 26490)
   5 : id_station_itineranc (56881 - 39857)
   6 : nom_station (69692 - 27046)
   7 : implantation_station (96733 - 5)
   8 : nbre_pdc (96674 - 64)
   9 : condition_acces (96736 - 2)
   10: horaires (96025 - 713)
   11: station_deux_roues (96732 - 6)
   12: id_pdc_itinerance (8137 - 88601)
   13: date_maj (95987 - 751)
   14: last_modified (96227 - 511)


-----------------------
## 4 - Bilan initial intégrité
- 25 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, schema)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          16251
contact_operateur - id_station_itinerance           9102
nom_enseigne - id_station_itinerance                9715
coordonneesXY - id_station_itinerance              10267
id_station_itinerance - id_pdc_itinerance           6683
nom_station - id_station_itinerance                10299
implantation_station - id_station_itinerance        4576
nbre_pdc - id_station_itinerance                    7067
condition_acces - id_station_itinerance               83
horaires - id_station_itinerance                      49
station_deux_roues - id_station_itinerance          9847
adresse_station - coordonneesXY                     5708

nombre d'enregistrements sans erreurs :  72222
nombre d'enregistrements avec au moins une erreur :  24516
taux d'erreur :  25  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 35% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  8137  soit  33  %
nombre de pdc sans doublon   :  16379  soit  67  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, schema)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            500
nom_enseigne - id_station_itinerance                 503
coordonneesXY - id_station_itinerance               1050
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1682
implantation_station - id_station_itinerance         356
nbre_pdc - id_station_itinerance                    3608
condition_acces - id_station_itinerance               31
horaires - id_station_itinerance                      30
station_deux_roues - id_station_itinerance          1266
adresse_station - coordonneesXY                     4081

nombre d'enregistrements sans erreurs :  7874
nombre d'enregistrements avec au moins une erreur :  8505


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 43 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1616  soit  7  %
nombre de pdc sans doublon   :  6889


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, schema)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                213
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  268
implantation_station - id_station_itinerance         195
nbre_pdc - id_station_itinerance                    3033
condition_acces - id_station_itinerance                4
horaires - id_station_itinerance                      27
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     1700

nombre d'enregistrements sans erreurs :  1416
nombre d'enregistrements avec au moins une erreur :  5473  soit  64  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(itinerance_5.npd.analysis().tree())

-1: root-derived (5473)
   0 : index (0 - 5473)
   1 : contact_operateur (5424 - 49)
   2 : nom_enseigne (5284 - 189)
   3 : coordonneesXY (4621 - 852)
   4 : adresse_station (4364 - 1109)
   5 : id_station_itineranc (3641 - 1832)
      13: date_maj (1734 - 98)
   6 : nom_station (4427 - 1046)
   7 : implantation_station (5468 - 5)
   8 : nbre_pdc (5439 - 34)
   9 : condition_acces (5471 - 2)
   10: horaires (5423 - 50)
   11: station_deux_roues (5467 - 6)
   12: id_pdc_itinerance (0 - 5473)
   14: last_modified (5390 - 83)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  11


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [ ]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

In [ ]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [ ]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

In [ ]:
# structure des données bonnes
#print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())
print(itinerance_ok.loc[:, mandatory].npd.analysis().tree())

### Indicateurs

In [ ]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [ ]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

In [ ]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

In [ ]:
# stockage des indicateurs
log |= indic
log_file = log['chemin'] + '/' + 'logfile.txt'
#with open('logfile.txt', 'a', encoding="utf-8") as f:
with open(log_file, 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [ ]:
maxi.loc[:, relations]

In [ ]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [ ]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print('\n', sum(itinerance_4.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

### Defauts operateur POWERDOT
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRPD1PCHAVDB' contient 28 points de recharge avec un nombre de points de rechage variable. Les identifiants des points de recharge et le nombre de PdC indiqué pour cet exemple montrent qu'il s'agit en réalité de 6 stations comportant chacune 3, 4, 8, 3, 8 et 2 PdC. Toutes les stations en défaut sont situées dans des parkings et concernent visiblement des stations implantées sur plusieurs étages et regroupées sous un même identifiant (on devrait donc a minima identifier une station par étage).

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'hello@powerdot.fr', relations][1000: 1500]

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRPD1PACCBEA', relations]

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRPD1PCHAVDB', relations]

### Defauts operateur ALIZECHARGE
Les défauts identifiés concernent des stations utilisant un même identifiant. Par exemple, la station 'FRADPP91479018' contient 82 points de recharge avec plusieurs adresses et un nombre de points de rechage variable. Cet exemple concerne des stations implantées sur plusieurs étages d'un parking (on devrait donc a minima identifier une station par étage).

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'support@alizecharge.fr', relations]

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRADPP91479018', relations]

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRTLSP31555040', relations]

### Defauts operateur IZIVIA
Les défauts au nombre de 280 sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Izivia en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance, ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur un parking [-0.530598, 47.398424], on trouve 17 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué entre 1 et 3.

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'sav@izivia.com', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-0.530598, 47.398424]', relations]

### Defauts operateur IONITY
Les défauts (480 points de recharge) sont liés à une incohérence entre adresse et coordonnées (une même coordonnée a plusieurs adresses différentes). Ce défaut est lié soit à des erreurs de saisie, soit à des ajouts (ou remplacements) ultérieurs de stations à une même localisation.

Les stations Ionity en erreur ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

Par exemple, sur l'aire de Mornas [4.732409, 44.194832], on trouve 26 stations avec la même coordonnée, le même identifiant entre point de recharge et station et un nombre de PdC indiqué de 23 pour chaque station.

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

In [ ]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.366838, 46.701436]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[2.523685, 48.9908]', relations]

### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [ ]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [ ]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

### Coherence adresse - coordonnees
- 1807 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [ ]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

In [ ]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

- exemple : Stations Ouest charge (double saisie par deux opérateurs)

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

### Cohérence station - coordonnées
- 323 pdc sont associés à des stations avec plusieurs coordonnées (dont 127 Freshmile)

In [ ]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][300:315]

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRLE2PYSCJHWOOBD', relations]

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance	 == 'FRFR1PEHNKRRPQNR', relations]

- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.08745032, 48.40887156]', relations]

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [ ]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [ ]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

In [ ]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [ ]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]